In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 11 #

## Lists

In [2]:
simple_list = ['hello', 7, 3.14, True]
simple_list

['hello', 7, 3.14, True]

In [3]:
my_array = make_array(1, 2, 3)

crowded_list = [my_array, 'what is going on', -10]
crowded_list

[array([1, 2, 3], dtype=int64), 'what is going on', -10]

In [4]:
Table().with_columns('Numbers', [1, 2, 3])

Numbers
1
2
3

In [5]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

Drink | Cafe | Price

In [6]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

Drink    | Cafe   | Price
Milk Tea | Asha   | 5.5
Espresso | Strada | 1.75
Latte    | Strada | 3.25
Espresso | FSM    | 2

## Grouping by one column

In [7]:
survey = Table.read_table('welcome_survey_v4.csv')
survey.show(3)

In [8]:
survey.group('Sleep position').show()

In [9]:
survey.group('Sleep position', np.average)

Sleep position     | Year average | Extraversion average | Number of textees average | Hours of sleep average | Handedness average
On your back       |              | 5.57947              | 6.60596                   | 7.11175                |
On your left side  |              | 5.66845              | 7.46791                   | 7.00535                |
On your right side |              | 5.89401              | 7.50922                   | 7.05645                |
On your stomach    |              | 5.96226              | 7.58491                   | 7.15094                |

In [10]:
survey.select('Sleep position', 'Hours of sleep').group('Sleep position', np.average)#removed leading (

Sleep position     | Hours of sleep average
On your back       | 7.11175
On your left side  | 7.00535
On your right side | 7.05645
On your stomach    | 7.15094

## Cross-classification: grouping by two columns

In [11]:
survey.group(['Handedness','Sleep position']).show()

In [12]:
survey.pivot('Sleep position', 'Handedness')

Handedness   | On your back | On your left side | On your right side | On your stomach
Both         | 4            | 5                 | 7                  | 1
Left-handed  | 31           | 27                | 30                 | 12
Right-handed | 267          | 342               | 397                | 199

In [13]:
survey.pivot('Sleep position', 'Handedness', 'Hours of sleep', np.average)

Handedness   | On your back | On your left side | On your right side | On your stomach
Both         | 6.125        | 6.6               | 7.42857            | 6
Left-handed  | 7.1371       | 7.11111           | 7.03333            | 6.70833
Right-handed | 7.1236       | 7.00292           | 7.05164            | 7.18342

In [14]:
(survey.select('Handedness', 'Sleep position', 'Hours of sleep')
       .group(['Handedness','Sleep position'], np.average)).show()

In [15]:
# Here, pivot doesn't know how to combine all the hours of sleep
# for each subgroup of students
survey.pivot('Sleep position', 'Handedness', 'Hours of sleep', 
             collect=np.average)#added collect

Handedness   | On your back | On your left side | On your right side | On your stomach
Both         | 6.125        | 6.6               | 7.42857            | 6
Left-handed  | 7.1371       | 7.11111           | 7.03333            | 6.70833
Right-handed | 7.1236       | 7.00292           | 7.05164            | 7.18342

## Challenge Question ##

In [16]:
sky = Table.read_table('skyscrapers_v2.csv')
sky = (sky.with_column('age', 2020 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

In [17]:
# 1. For each city, what’s the tallest building for each material?










In [18]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?












Don't read ahead until you try the challenge questions yourself first!

In [19]:
sky.select('material', 'city', 'height').group(['city', 'material'], max)

city      | material        | height max
Atlanta   | concrete        | 264.25
Atlanta   | mixed/composite | 311.8
Atlanta   | steel           | 169.47
Austin    | concrete        | 208.15
Austin    | steel           | 93.6
Baltimore | concrete        | 161.24
Baltimore | steel           | 155.15
Boston    | concrete        | 121.92
Boston    | mixed/composite | 139
Boston    | steel           | 240.79
... (81 rows omitted)

In [20]:
sky_p = sky.pivot('material', 'city', 'height', max)
sky_p.show()

In [21]:
sky_p = sky_p.with_column(
    'difference', 
    abs(sky_p.column('steel') - sky_p.column('concrete'))
)
sky_p

city       | concrete | mixed/composite | steel  | difference
Atlanta    | 264.25   | 311.8           | 169.47 | 94.78
Austin     | 208.15   | 0               | 93.6   | 114.55
Baltimore  | 161.24   | 0               | 155.15 | 6.09001
Boston     | 121.92   | 139             | 240.79 | 118.87
Charlotte  | 265.48   | 239.7           | 179.23 | 86.25
Chicago    | 423.22   | 306.94          | 442.14 | 18.92
Cincinnati | 125      | 202.69          | 175    | 50
Cleveland  | 125      | 288.65          | 215.8  | 90.8
Columbus   | 79.25    | 0               | 169.3  | 90.05
Dallas     | 176.48   | 280.72          | 270.06 | 93.58
... (25 rows omitted)

In [22]:
sky_p.sort('difference', True)

city              | concrete | mixed/composite | steel  | difference
Sunny Isles Beach | 196      | 0               | 0      | 196
Las Vegas         | 350.22   | 195.68          | 164.6  | 185.62
Miami Beach       | 170.39   | 0               | 0      | 170.39
Pittsburgh        | 89.3     | 172             | 256.34 | 167.04
Los Angeles       | 145.7    | 118.26          | 310.29 | 164.59
Philadelphia      | 157.89   | 296.73          | 288.04 | 130.15
Boston            | 121.92   | 139             | 240.79 | 118.87
Austin            | 208.15   | 0               | 93.6   | 114.55
Seattle           | 138.69   | 284.38          | 235.31 | 96.62
Atlanta           | 264.25   | 311.8           | 169.47 | 94.78
... (25 rows omitted)

### Take-home question: try it here!

In [23]:
# Generate a table of the names of the oldest buildings for each 
# material for each city:



## Joins ##

In [24]:
drinks

Drink    | Cafe   | Price
Milk Tea | Asha   | 5.5
Espresso | Strada | 1.75
Latte    | Strada | 3.25
Espresso | FSM    | 2

In [25]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

Coupon % off | Location
10           | Asha
25           | Strada
5            | Asha

In [26]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

Cafe   | Drink    | Price | Coupon % off
Asha   | Milk Tea | 5.5   | 10
Asha   | Milk Tea | 5.5   | 5
Strada | Espresso | 1.75  | 25
Strada | Latte    | 3.25  | 25

In [27]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

Cafe   | Drink    | Price | Coupon % off | Discounted Price
Asha   | Milk Tea | 5.5   | 10           | 4.95
Asha   | Milk Tea | 5.5   | 5            | 5.225
Strada | Espresso | 1.75  | 25           | 1.3125
Strada | Latte    | 3.25  | 25           | 2.4375

In [28]:
drinks.join('Cafe', drinks, 'Cafe')

Cafe   | Drink    | Price | Drink_2  | Price_2
Asha   | Milk Tea | 5.5   | Milk Tea | 5.5
FSM    | Espresso | 2     | Espresso | 2
Strada | Espresso | 1.75  | Espresso | 1.75
Strada | Espresso | 1.75  | Latte    | 3.25
Strada | Latte    | 3.25  | Espresso | 1.75
Strada | Latte    | 3.25  | Latte    | 3.25